In [1]:
import pandas as pd
import numpy as np
from numpy import random as r
from matplotlib import pyplot as plt

In [ ]:
import Quandl as q

In [35]:
#create a sample
path="C:\\dev\\psignal\\data\\"
pureTwt='twitter_noretweets_daily.csv'
ptwt=pd.read_csv(filepath_or_buffer=path+pureTwt,delimiter=",",header=0)
ptwt=ptwt.sample(n=1000)
ptwt['TIMESTAMP_UTC']=pd.to_datetime(arg=ptwt.TIMESTAMP_UTC,infer_datetime_format='true',unit='D')
#ptwt['TIMESTAMP_UTC']=pd.to_datetime(arg=ptwt.TIMESTAMP_UTC,format='%Y%m%d')
cols=['SYMBOL','TIMESTAMP_UTC']
ptwt=ptwt.reindex(columns=cols)
ptwt.columns=['sym','date']
ptwt['cluster']=r.randint(low=1,high=5,size=len(ptwt))
ptwt.to_csv("sample ptwt.csv")
#this is how data should look like after clustring
ptwt.head()

,sym,date,cluster
3551142,NUAN,2012-04-04 04:00:00,3
667150,BOH,2013-03-22 04:00:00,2
1767276,EWJ,2013-04-21 04:00:00,4
855904,CCMP,2013-01-01 05:00:00,2
2092424,GED,2014-04-28 04:00:00,3


In [32]:
#if sample already exists use this to import it
sampleTWT='sample ptwt.csv'
ptwt=pd.read_csv(filepath_or_buffer=path+sampleTWT,delimiter=",",header=0)

AttributeError: 'Series' object has no attribute 'set_freq'

In [ ]:
#data is imported from clustering stage
data = pd.csv_read('../data/data2010to2015.csv')
#format is: date | sym  | cluster
#date is daily
#sym is cashtag/ticker
#cluster is a number that indicates cluster

tickers = data['sym'].unique()
rdata=pd.DataFrame({})
for sym in tickers:   
    try:
        #can later expand this to include search from other db than WIKI
        tmp=q.get("WIKI/"+sym+".11",start='2010-01-01', end='2016-01-01', transformation='rdiff')
        tmp[sym]=sym
    except:
        tmp= #NaNs dataframe
    rdata=pd.concat(rdata,tmp)


mi = pd.read_csv('../data/sp500.csv')

data = data.set_index(data['date'])

mi = mi.set_index(pd.to_datetime(mi['Date']))


In [54]:
a=pd.DataFrame({})
b=pd.DataFrame({'a':['2','3']})
pd.concat(a,b)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [126]:
sp500=pd.DataFrame({'date':pd.date_range(start='2010-01-01',end='2015-12-31',frequency="D"),\
                    'return':r.randn(len(pd.date_range(start='2010-01-01',end='2015-12-31',frequency="D")))})
sp500.head()

,date,return
0,2010-01-01,-0.078026
1,2010-01-02,0.921009
2,2010-01-03,-0.569918
3,2010-01-04,0.352940
4,2010-01-05,1.514005


In [120]:
windows=pd.period_range(start='2010',end='2015',freq="M")
windows=windows.format()
windows[0]

'2010-01'

In [ ]:
import <regression> as reg
for stock in ptwt['sym'].unique():
    for window_end_idx in range(1,len(windows)):
        mask=(ptwt['sym']==stock)&(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        tmp=ptwp[mask]
        r_t_plus_one=tmp['return'].ix[1:]
        r_t=tmp['return'].ix[0:len(tmp)-1]
        mask_market_index=(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        market_index=SP500[mask].ix[0:len(tmp)-1] #check and correct for stationarity
        SIR=ptwt[mask_market_index].groupby('cluster','date').mean() #compute mean return for each day and cluster
        stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        while (stationarity_test):
            r_t_plus_one=reg.takeFirstDifference(r_t_plus_one)
            r_t=reg.takeFirstDifference(r_t)
            stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        tmp_reg=reg.GLS(y=r_t_plus_one,x=[r_t,market_index],include_constant=True,return_residuals=True)
        tmp_reg_prime=reg.GLS(y=tmp_reg.residuals,x=[r_t,SIR],include_constant=True,return_residuals=True)
        regression[(stock, window)]=(tmp_reg,tmp_reg_prime)

In [122]:
mask=(ptwt['sym']=='ALE')&(ptwt['date']>='2015-1')&(ptwt['date']<='2015-12')
ptwt[mask]

,sym,date,return
3,ALE,2015-01-16 05:00:00,1.54638
